## Comparing methods of calculating weighted regional and global means

### Import packages

In [ ]:
import numpy as np
import xarray as xr

import esmlab
import matplotlib.pyplot as plt

### Load in some test data

In [ ]:
scr_dir = "/glade/scratch/kdagon/archive/"
case_name = "hydro_ensemble_default_params"
file_path = scr_dir + case_name + "/lnd/hist/"
print(file_path)

Specify model year to read in

In [ ]:
model_yr = 30

In [ ]:
fils_in = xr.open_mfdataset(
    file_path + "*.clm2.h0.00" + str(model_yr) + "*.nc", combine="by_coords"
)

In [ ]:
fils_in

### Pull out a specific output variable

In [ ]:
TSA = fils_in.TSA  # this is now a DataArray
TSA

### Gridcell area and landfrac info (first time index)

In [ ]:
area = fils_in.area[0]
landfrac = fils_in.landfrac[0]
landfrac

### Calculate area weights

In [ ]:
arwt = area * landfrac

In [ ]:
# This doesn't work - how to reassign long_name to arwt?
# arwt.long_name = 'grid cell areas multiplied by landfrac'

In [ ]:
arwt.plot(robust=True)

Normalize arwt: 2 methods!

In [ ]:
arwt_norm_sum = arwt / arwt.sum()

In [ ]:
arwt_norm_max = arwt / arwt.max()

In [ ]:
arwt_norm_max

## TO DO: calc area weights (arwt) with 2 different normalizations, esmlab weighted means function, cos(latitude) weighting, other NCL/tutorial methods?
### Test global and regional both methods